# Project to AKS

## Reference 

- https://homel.vsb.cz/~vas218/pdf/acs/grammar.pdf
- https://homel.vsb.cz/~vas218/pdf/acs/vasinek-thesis.pdf
- https://homel.vsb.cz/~vas218/acs.html

## Choosen task

- Reduction paradox

## Description

- Try to reduce message and with every step maximaze entropy

## Official description 

- **RePair - maximal anticompression - reduction paradox**
    - Find the smallest possible representation(measured in the number of symbols) of file using the reduction paradox which leads to the largest increase of zero order entropy representation.
    - Heuristics - largest first, random
    - Describe the algorithm and summarize results to a .doc(x) or .pdf report.
    - Prepare a presentation for 10 minutes about your method.
    - Literature: Vasinek, Dissertation thesis (Chapter 5)

## Steps

- Load dataset
- Can pick a subset for faster running (sample it)
- Find every bigram and try to create rule
- Calculate Entropy for new message
- Need to find rule which reduces size of message but increases entropy (harder to compress)
- Find extremes for every file
- Show graphs

In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*1))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [2]:
from src.load_data import get_dataset
from src.load_data import DataSets
from src.get_probs import get_sorted_probs_as_df
import numpy as np
import pandas as pd
import time
from src.save import save_both
from enum import Enum
import re
from ast import literal_eval
import random
import plotly.express as px


In [3]:
TEST_NORMALIZATION_SIZE = 10000

In [4]:
def load_dataset(type, normalization=None):
    data_dna, path_dna = get_dataset(type)
    if normalization is None:
        return data_dna, path_dna
    return "".join(np.random.choice(list(data_dna), normalization)), path_dna

In [5]:
def get_datasets(normalization=None):
    return [
        #(*get_dataset(DataSets.english, normalization), DataSets.english), 
        (*load_dataset(DataSets.dna, normalization), DataSets.dna),
        #(*load_dataset(DataSets.proteins, normalization), DataSets.proteins),
        #(*load_dataset(DataSets.sources, normalization), DataSets.sources),
    ]


In [6]:
test_datasets = get_datasets(TEST_NORMALIZATION_SIZE)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB


In [7]:
test_datasets

[('CCCGATATGCATATGTCATGGTCCTGGATTTACAAGGATCGAGTAATGGCAACTGTCTGTACTGATATTTCCAATTTCTTAAGTATTTCGTGTTAAAGCATGGATGTAAGGTTGAGTTATATGGGTATATGCAACATATCATAATTGGAAGTATCAGCGTAATCTCTATAATAGTAAACATCTCAGTATCATGCCTCTTAAGCTATATTCGTCCGACCTATCCGTAAGATTCATATTTATAGCGTGTGCCTGCGGACGATCAAATGGGAGAGGTTACCTCGTGATTCCAGCCGCTATATTAGGTGTATTCCTTTCGTACCCGAACTCGTGATCCCTAGGAAGAGATTATCGACTGTACGATAAGGTGCGAGAGTGAAACAAAGCGCGTTTATCACTACTTAATGATGCGCACAATTCACACTATGACGCACCGCGTCATACAAGAATTTATGCCCTATCCTGATAACTATTTTTCAATGCGATTTTGAAATGACCATATCGCATTTCTAAACTAATCTGACGAGGGCTATCTATGTTCTTTGCAATGTCCACCGTTCTACGACTCACTGCGAACTCCAATTCTACCAAGAGGTCTGAAACACCTAAATTCCTGCATTACGGTCTTAGGATTTCTTACTTATGGATGTATCAGTCACATATAGAGTAATAGGGTTAAATTTGGTATGATTTACGTGTGAAATCCAAGTTGATGGACATCCGGAGGTCGTCAGTTGCCTGAGGGTCCATGAATCAATAATCTCTATTTCGAGCCTCCCTTTCGTATTAGAGTTAGATCAATTCTTTAACCGCTTGTCGGCTTCACCGTAGTCGAGTTTAGGGGGTGGGGACGGCAACCCGAACTTAGTCCGATTTCACGGGACTTGTTGCGTACTAAGATCTTACGTCTATGAAGAACGCGGTAATTAGTTTTCCACGCATGCTGACTATATACATCCTGGCACTTAACAATCGTGTATGACGGGACGGTTTTTTGAAT

In [8]:
data_test, data_path, data_type = test_datasets[0]

In [9]:
def find_k_grams_freq(data, max_size_k=2):
    
    kgrams_dic = {}

    for k in range(2, max_size_k+1):

        for i in range(len(data) - k):

            n_gram = data[i:i+k]
            
            kgrams_dic[n_gram] = kgrams_dic.get(n_gram, 0) + 1

    return kgrams_dic

In [10]:
find_k_grams_freq(data_test)

{'CC': 407,
 'CG': 433,
 'GA': 577,
 'AT': 866,
 'TA': 914,
 'TG': 600,
 'GC': 444,
 'CA': 584,
 'GT': 604,
 'TC': 601,
 'GG': 402,
 'CT': 645,
 'TT': 850,
 'AC': 617,
 'AA': 861,
 'AG': 591,
 'CN': 1,
 'NG': 1}

In [11]:
import math
from collections import Counter

def calc_freq(content):
    c = Counter(list(content))
    return c

def calc_p(counter, n):
    counter = dict(counter)
    res = {}
    for k, v in counter.items():
        res[k] = v / n  
    return res

def get_n(counter):
    counter = dict(counter)
    return np.sum(list(counter.values()))

def calc_H(p):
    H = 0
    for k, v in p.items():
        #Shannon equation!
        H += p[k] * math.log2(p[k])
    return -H

def calc_entropy_for_message(message):
    counter = calc_freq(message)
    n = get_n(counter)
    p = calc_p(counter, n)
    H = calc_H(p)
    return H

In [12]:
calc_entropy_for_message(data_test)

1.9776479278504808

In [13]:
def diff_entropy(message1, message2):
    message1_entropy = calc_entropy_for_message(message1)
    message2_entropy = calc_entropy_for_message(message2)
    message_1_entropy_size = message1_entropy * len(message1)
    message_2_entropy_size = message2_entropy * len(message2)


    diff = message_1_entropy_size - message_2_entropy_size
    #print(f'{round(message1_entropy, 2)} * {len(message1)} < {round(message2_entropy, 2)} * {len(message2)} ... {diff}')
    return diff

In [14]:
diff_entropy(data_test, data_test)

0.0

In [15]:
all_chars_uni = tuple(chr(i) for i in range(32, 0x110000) if chr(i).isprintable())

In [16]:
all_chars_ascii = list(range(0, 256))
all_chars_ascii = [chr(ascii_char) for ascii_char in all_chars_ascii]

In [17]:
def find_not_existing_character(current_alpahbet, gen_chars=all_chars_uni):
    can_use = set(gen_chars).difference(set(current_alpahbet))
    ascii_picked_char = random.choice(list(can_use))
    return ascii_picked_char

In [18]:
find_not_existing_character(['A', 'C', 'G', 'T'])

'𛇮'

In [19]:
def transform_message(message, ngram_for_replace):
    current_alphabet_size = np.unique(list(message))

    #print('Alphabet size', len(current_alphabet_size))

    replace_character = find_not_existing_character(current_alphabet_size)
    
    return message.replace(ngram_for_replace, replace_character), replace_character

In [20]:
def calculate_for_ngrams_diff(ngrams, message, method_entropy=diff_entropy, init_message=None):
    res = {}
    
    for k, v in ngrams.items():
        res[k] = {
            "Counter": v,
            #message - current
            #message - next message
            #"Diff": method_entropy(init_message, transform_message(message, k)[0])
            "Diff": method_entropy(message, transform_message(message, k)[0])
        }
    return res

In [21]:
r = calculate_for_ngrams_diff(
        find_k_grams_freq(data_test),
        data_test,
        diff_entropy,
        data_test
    )

pd.DataFrame.from_dict(
    r, 
    orient="index"
)

,Counter,Diff
CC,407,-429.811368
CG,433,-476.602214
GA,577,-663.262153
AT,866,-902.599147
TA,914,-857.800771
TG,600,-656.357127
GC,444,-468.978055
CA,584,-679.783285
GT,604,-653.362705
TC,601,-677.653314


In [22]:
r

{'CC': {'Counter': 407, 'Diff': -429.81136813989724},
 'CG': {'Counter': 433, 'Diff': -476.6022139988163},
 'GA': {'Counter': 577, 'Diff': -663.2621527918345},
 'AT': {'Counter': 866, 'Diff': -902.599147238976},
 'TA': {'Counter': 914, 'Diff': -857.8007706875178},
 'TG': {'Counter': 600, 'Diff': -656.3571271993169},
 'GC': {'Counter': 444, 'Diff': -468.97805533048813},
 'CA': {'Counter': 584, 'Diff': -679.7832854468397},
 'GT': {'Counter': 604, 'Diff': -653.3627048300841},
 'TC': {'Counter': 601, 'Diff': -677.6533135408063},
 'GG': {'Counter': 402, 'Diff': -407.50893022703895},
 'CT': {'Counter': 645, 'Diff': -643.139569584946},
 'TT': {'Counter': 850, 'Diff': -743.8652911470417},
 'AC': {'Counter': 617, 'Diff': -655.3523424227278},
 'AA': {'Counter': 861, 'Diff': -728.7446691381738},
 'AG': {'Counter': 591, 'Diff': -653.5157959937023},
 'CN': {'Counter': 1, 'Diff': 2.2718767690494133},
 'NG': {'Counter': 1, 'Diff': 2.302865796984406}}

In [23]:
r.items()

dict_items([('CC', {'Counter': 407, 'Diff': -429.81136813989724}), ('CG', {'Counter': 433, 'Diff': -476.6022139988163}), ('GA', {'Counter': 577, 'Diff': -663.2621527918345}), ('AT', {'Counter': 866, 'Diff': -902.599147238976}), ('TA', {'Counter': 914, 'Diff': -857.8007706875178}), ('TG', {'Counter': 600, 'Diff': -656.3571271993169}), ('GC', {'Counter': 444, 'Diff': -468.97805533048813}), ('CA', {'Counter': 584, 'Diff': -679.7832854468397}), ('GT', {'Counter': 604, 'Diff': -653.3627048300841}), ('TC', {'Counter': 601, 'Diff': -677.6533135408063}), ('GG', {'Counter': 402, 'Diff': -407.50893022703895}), ('CT', {'Counter': 645, 'Diff': -643.139569584946}), ('TT', {'Counter': 850, 'Diff': -743.8652911470417}), ('AC', {'Counter': 617, 'Diff': -655.3523424227278}), ('AA', {'Counter': 861, 'Diff': -728.7446691381738}), ('AG', {'Counter': 591, 'Diff': -653.5157959937023}), ('CN', {'Counter': 1, 'Diff': 2.2718767690494133}), ('NG', {'Counter': 1, 'Diff': 2.302865796984406})])

In [24]:
def pick_largest(items):
    return list(sorted(items, key=lambda x: x[1]['Diff']))[0]

def pick_random(items):
    return random.choice(items)

def pick_only_decreasing(dic, pick_method=pick_largest):
    items = dic.items()

    decreasing_items = list(filter(lambda x: x[1]['Diff'] < 0, items))

    if len(decreasing_items) == 0:
        return None

    return pick_method(decreasing_items)    

In [25]:
pick_only_decreasing(r, pick_largest)

('AT', {'Counter': 866, 'Diff': -902.599147238976})

# Algorithm implementation - Reduction paradox

In [26]:
class TableFields(Enum):
    Rule = "Rule"
    EntropyMove = "EntropyMove"
    CurrentEntropy = "CurrentEntropy"
    EntropySize = "EntropySize"
    DataType = "DataType"
    DescriptionData = "DescriptionData"
    MessageSize = "MessageSize"
    AlphabetSize = "AlphabetSize"
    GrammaticSize = "GrammaticSize"
    CalcTime = "CalcTime"


In [27]:
def create_value(message_0, message_1, diff, replace_character, n_gram, grammatic, tic, type_data=None, description_data=None):
    new_message_alphabet_size = len(np.unique(list(message_1)))
    new_message_size = len(message_1)
    new_message_entropy = calc_entropy_for_message(message_1)

    new_rule = f"{n_gram} -> {replace_character}"
    grammatic[n_gram] = replace_character

    tac = time.time()
    
    return {
        TableFields.Rule.value: new_rule,
        TableFields.EntropyMove.value: diff,
        TableFields.CurrentEntropy.value: new_message_entropy,
        TableFields.EntropySize.value: new_message_entropy * new_message_size,
        TableFields.DataType.value: type_data,
        TableFields.DescriptionData.value: description_data,
        TableFields.MessageSize.value: new_message_size,
        TableFields.AlphabetSize.value: new_message_alphabet_size,
        TableFields.GrammaticSize.value: len(list(grammatic.keys())),
        TableFields.CalcTime.value: tac - tic,
    }


In [28]:
def algorithm_step(init_message, message, grammatic, type_data=None, description_data=None, tic=None, heuristics_method=pick_largest, optimized=False):
    #Find ngrams
    n_grams = find_k_grams_freq(message)
    
    #optimized = diff_entropy change pls use formula

    diff_table = calculate_for_ngrams_diff(
        n_grams,
        message,
        diff_entropy,
        init_message
    )

    picked = pick_only_decreasing(diff_table, heuristics_method)
    print(picked)

    if picked is None:
        return None

    #('AT', {'Counter': 867, 'Diff': -0.2814694848677286})
    n_gram, dic_values = picked
    transformed_message, replace_character = transform_message(message, n_gram)
    
    return transformed_message, create_value(message, transformed_message, dic_values['Diff'], replace_character, n_gram, grammatic, tic, type_data, description_data)


In [46]:
def algorithm(message, type_data=None, description_data=None, limit_step=None, heuristic_method=pick_largest, optimized=False):
    init_message = message

    res = {}
    grammatic = {}
    step = 0
#message_0, message_1, diff, replace_character, n_gram, grammatic, tic, type_data=None, description_data=None
    step_value = create_value(init_message, init_message, 0, "", "", grammatic, time.time(), type_data)

    res[step] = step_value
    step += 1

    while True:
        tic = time.time()
        if limit_step is not None and limit_step == step:
            break
        step_value = algorithm_step(init_message, message, grammatic, type_data, description_data, tic, heuristic_method, optimized)
        print(step, len(message))
        print('\n')
        if step_value is None:
            break

        else:
            transformed_message, value = step_value
            message = transformed_message

            step += 1
            res[step] = value

    return res

## Testing data

In [47]:
test_result = algorithm(data_test, limit_step=20)

('AT', {'Counter': 866, 'Diff': -902.599147238976})
1 10000


('TC', {'Counter': 421, 'Diff': -564.2114557037385})
2 9134


('GA', {'Counter': 408, 'Diff': -574.2843182809556})
3 8713


('CG', {'Counter': 273, 'Diff': -395.204272314666})
4 8305


('CA', {'Counter': 329, 'Diff': -303.967591233999})
5 8032


('TG', {'Counter': 359, 'Diff': -289.3102175666063})
6 7703


('𑄨T', {'Counter': 166, 'Diff': -186.2967022102166})
7 7344


('AC', {'Counter': 248, 'Diff': -194.63337510700148})
8 7178


('GT', {'Counter': 240, 'Diff': -136.14568108932508})
9 6929


('𑄨A', {'Counter': 143, 'Diff': -123.82819112846119})
10 6689


('T𑄨', {'Counter': 86, 'Diff': -96.12848366744583})
11 6546


('𮊳A', {'Counter': 68, 'Diff': -76.46490543075925})
12 6460


('C𑄨', {'Counter': 65, 'Diff': -74.25959822794175})
13 6392


('A♂', {'Counter': 55, 'Diff': -73.38207233108187})
14 6327


('T♂', {'Counter': 51, 'Diff': -59.2440128585622})
15 6272


('G𑄨', {'Counter': 60, 'Diff': -58.699002428933454})
16 6221


('CC',

In [48]:
df = pd.DataFrame.from_dict(test_result, orient="index")
df

,Rule,EntropyMove,CurrentEntropy,EntropySize,DataType,DescriptionData,MessageSize,AlphabetSize,GrammaticSize,CalcTime
0,->,0.000000,1.977648,19776.479279,None,None,10000,5,1,0.003001
2,AT -> 𑄨,-902.599147,2.263967,20679.078426,None,None,9134,6,2,0.487568
3,TC -> 𮊳,-564.211456,2.438114,21243.289881,None,None,8713,7,3,0.650164
4,GA -> ♂,-574.284318,2.627041,21817.574200,None,None,8305,8,4,0.857145
5,CG -> 脮,-395.204272,2.765535,22212.778472,None,None,8032,9,5,1.072077
6,CA -> 𨽴,-303.967591,2.923114,22516.746063,None,None,7703,10,6,1.360998
7,TG -> 홄,-289.310218,3.105400,22806.056281,None,None,7344,11,7,1.781959
8,𑄨T -> 𪇭,-186.296702,3.203170,22992.352983,None,None,7178,12,8,1.984849
9,AC -> 湛,-194.633375,3.346368,23186.986358,None,None,6929,13,9,2.590003
10,GT -> 𣟂,-136.145681,3.486789,23323.132039,None,None,6689,14,10,2.746336


In [49]:
fig = px.line(df, x=df.index, y=df.EntropyMove, text=[("%.1f" % x) for x in df.EntropyMove.values], title='Pohyb entropie')
fig.show()

In [50]:
fig = px.line(df, x=df.index, y=df.MessageSize, text=df.MessageSize, title='Velikost zprávy')
fig.show()
fig.write_image("test.png")

In [51]:
fig = px.line(df, x=df.index, y=[df.MessageSize, df.EntropySize], title='Velikost zprávy proti aktuální entropii')
fig.show()

In [52]:
fig = px.line(df, x=df.index, y=df.EntropySize, title='H*len(m)', text=[("%.0f" % x) for x in df.EntropySize.values])
fig.update_traces(textposition='top center')
fig.show()

# Real experiment data

In [53]:
def get_datasets(normalization=None):
    return [
        #(*load_dataset(DataSets.english, normalization), DataSets.english), 
        (*load_dataset(DataSets.dna, normalization), DataSets.dna),
        #(*load_dataset(DataSets.proteins, normalization), DataSets.proteins),
        #(*load_dataset(DataSets.sources, normalization), DataSets.sources),
    ]

In [38]:
NORM_VALUES = [10000]
CSV_NAME = "steps.csv"
ENTROPY_GRAPH = "entropy_paradox.png"
MESSAGE_GRAPH = "message_size.png"
MESSAGE_SIZE_ENTROPY_GRAPH = "message_entropy_size.png"
MESSAGE_ENTROPY = "message_entropy.png"

In [32]:
def write_images(df, path):
    fig = px.line(df, x=df.index, y=df.EntropyMove, text=[("%.1f" % x) for x in df.EntropyMove.values], title='Pohyb entropie')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, ENTROPY_GRAPH]))
    
    fig = px.line(df, x=df.index, y=df.MessageSize, text=df.MessageSize, title='Velikost zprávy')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, MESSAGE_GRAPH]))    

    fig = px.line(df, x=df.index, y=[df.MessageSize, df.EntropySize], title='Velikost zprávy proti aktuální entropii')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, MESSAGE_SIZE_ENTROPY_GRAPH]))

    fig = px.line(df, x=df.index, y=df.EntropySize, title='H*len(m)', text=[("%.0f" % x) for x in df.EntropySize.values])
    fig.update_traces(textposition='top center')   
    fig.write_image(os.path.sep.join([path, MESSAGE_ENTROPY]))

In [33]:
def save_dataframe(df, path):
    path = os.path.sep.join([path, CSV_NAME])
    df.to_csv(path, index=False)

In [34]:
def run_algorithm_for_datasets(normalization_values=NORM_VALUES, largest=True, limit_steps=None, optimized=False):
    for n_v in normalization_values:

        datasets = get_datasets(n_v)

        for data, data_path, data_type in datasets:
            data_type_string = data_type.value
            current_path = os.path.sep.join([data_type_string, str(n_v)])


            limit_steps_str = "None" if limit_steps is None else str(limit_steps)
            optimized_string = "optimized" if optimized else "bruto"

            steps_path = os.path.sep.join([current_path, limit_steps_str, str(largest), optimized_string])

            if not os.path.isdir(steps_path):
                os.makedirs(steps_path)

            heurestic_method = pick_largest if largest else pick_random

            res = algorithm(data, data_type_string, "", limit_step=limit_steps, heuristic_method=heurestic_method, optimized=optimized)
            df = pd.DataFrame.from_dict(res, orient="index")




            save_dataframe(df, steps_path)
            write_images(df, steps_path)

In [35]:
LIMIT_STEPS = [None]
LARGEST_METHOD = [True] #largest || random
OPTIMIZED = [False] #not optimized (bruto) || optimized

In [36]:
import itertools

In [37]:
exps = list(itertools.product(LIMIT_STEPS, LARGEST_METHOD, OPTIMIZED))
exps

[(None, True, False)]

In [39]:
for limit_steps, largest_method, optimized in exps:
    run_algorithm_for_datasets(NORM_VALUES, largest_method, limit_steps, optimized)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB


KeyboardInterrupt: 